# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [26]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import re
from datetime import datetime, timedelta
import numpy as np

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

In [5]:
# Generate the date range for Yellow Taxi and HVFHV datasets
start_date = "2009-01"
end_date = "2024-08"

dates = []
current_date = datetime.strptime(start_date, "%Y-%m")
end_date_obj = datetime.strptime(end_date, "%Y-%m")

while current_date <= end_date_obj:
    dates.append(current_date.strftime("%Y-%m"))
    current_date += timedelta(days=31)  # Move to the next month
    current_date = current_date.replace(day=1)

# Base URLs for Yellow Taxi and HVFHV datasets
yellow_taxi_base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"
hvhf_base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_"

# Save directories for Yellow Taxi and HVFHV data
yellow_taxi_save_dir = "data/yellow_taxi"
hvhf_save_dir = "data/hvhf"

# Ensure the directories exist
os.makedirs(yellow_taxi_save_dir, exist_ok=True)
os.makedirs(hvhf_save_dir, exist_ok=True)

# Download Yellow Taxi data
for date in dates:
    file_url = f"{yellow_taxi_base_url}{date}.parquet"
    file_name = f"{date}.parquet"
    local_file_path = os.path.join(yellow_taxi_save_dir, file_name)

    print(f"Downloading Yellow Taxi file: {file_url} ...")
    try:
        response = requests.get(file_url, stream=True)
        response.raise_for_status()
        with open(local_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"File saved to: {local_file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_url}: {e}")


File saved to: data/yellow_taxi\2009-01.parquet
File saved to: data/yellow_taxi\2009-02.parquet
File saved to: data/yellow_taxi\2009-03.parquet
File saved to: data/yellow_taxi\2009-04.parquet
File saved to: data/yellow_taxi\2009-05.parquet
File saved to: data/yellow_taxi\2009-06.parquet
File saved to: data/yellow_taxi\2009-07.parquet
File saved to: data/yellow_taxi\2009-08.parquet
File saved to: data/yellow_taxi\2009-09.parquet
File saved to: data/yellow_taxi\2009-10.parquet
File saved to: data/yellow_taxi\2009-11.parquet
File saved to: data/yellow_taxi\2009-12.parquet
File saved to: data/yellow_taxi\2010-01.parquet
File saved to: data/yellow_taxi\2010-02.parquet
File saved to: data/yellow_taxi\2010-03.parquet
File saved to: data/yellow_taxi\2010-04.parquet
File saved to: data/yellow_taxi\2010-05.parquet
File saved to: data/yellow_taxi\2010-06.parquet
File saved to: data/yellow_taxi\2010-07.parquet
File saved to: data/yellow_taxi\2010-08.parquet
File saved to: data/yellow_taxi\2010-09.

In [14]:
# Adjusted date range for HVFHV dataset (starting from February 2019)
start_date = "2019-02"
end_date = "2024-08"

# Generate the list of dates for HVFHV (starting from February 2019)
dates = []
current_date = datetime.strptime(start_date, "%Y-%m")
end_date_obj = datetime.strptime(end_date, "%Y-%m")

while current_date <= end_date_obj:
    dates.append(current_date.strftime("%Y-%m"))
    current_date += timedelta(days=31)  # Move to the next month
    current_date = current_date.replace(day=1)

# Base URL for HVFHV dataset
hvhf_base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_"

# Save directory for HVFHV data
hvhf_save_dir = "data/hvhf"

# Ensure the directory exists
os.makedirs(hvhf_save_dir, exist_ok=True)

# Download HVFHV data starting from February 2019
for date in dates:
    file_url = f"{hvhf_base_url}{date}.parquet"
    file_name = f"{date}.parquet"
    local_file_path = os.path.join(hvhf_save_dir, file_name)

    print(f"Downloading HVFHV file: {file_url} ...")
    try:
        response = requests.get(file_url, stream=True)
        response.raise_for_status()
        with open(local_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"File saved to: {local_file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_url}: {e}")

File saved to: data/hvhf\2019-02.parquet
File saved to: data/hvhf\2019-03.parquet
File saved to: data/hvhf\2019-04.parquet
File saved to: data/hvhf\2019-05.parquet
File saved to: data/hvhf\2019-06.parquet
File saved to: data/hvhf\2019-07.parquet
File saved to: data/hvhf\2019-08.parquet
File saved to: data/hvhf\2019-09.parquet
File saved to: data/hvhf\2019-10.parquet
File saved to: data/hvhf\2019-11.parquet
File saved to: data/hvhf\2019-12.parquet
File saved to: data/hvhf\2020-01.parquet
File saved to: data/hvhf\2020-02.parquet
File saved to: data/hvhf\2020-03.parquet
File saved to: data/hvhf\2020-04.parquet
File saved to: data/hvhf\2020-05.parquet
File saved to: data/hvhf\2020-06.parquet
File saved to: data/hvhf\2020-07.parquet
File saved to: data/hvhf\2020-08.parquet
File saved to: data/hvhf\2020-09.parquet
File saved to: data/hvhf\2020-10.parquet
File saved to: data/hvhf\2020-11.parquet
File saved to: data/hvhf\2020-12.parquet
File saved to: data/hvhf\2021-01.parquet
File saved to: d

In [30]:
def filter_uber_rides(hvhf_data: pd.DataFrame) -> pd.DataFrame:
    """
    Filters out non-Uber rides from the HVFHV dataset.

    Args:
        hvhf_data (pd.DataFrame): The raw HVFHV dataset.

    Returns:
        pd.DataFrame: Filtered dataset containing only Uber rides.
    """
    # Ensure 'Hvfhs_license_num' is treated as a string
    hvhf_data['hvfhs_license_num'] = hvhf_data['hvfhs_license_num'].astype(str)
    
    # Filter rows where 'hvfhs_license_num' is 'HV0003' (Uber)
    uber_only_data = hvhf_data[hvhf_data['hvfhs_license_num'] == 'HV0003'].copy()

    print(f"Filtered Uber rides: {len(uber_only_data)} out of {len(hvhf_data)} total rides.")
    
    # Warning if no Uber rides are found
    if uber_only_data.empty:
        print("Warning: No Uber rides found after filtering.")
    
    return uber_only_data


def cochran_sample_size(population_size: int, confidence_level: float = 0.95, margin_of_error: float = 0.05, p: float = 0.5) -> int:
    """
    Calculate the sample size using Cochran's formula.

    Args:
        population_size (int): The total number of data points in the population.
        confidence_level (float): The confidence level (default is 0.95).
        margin_of_error (float): The margin of error (default is 0.05).
        p (float): The estimated proportion of the population.

    Returns:
        int: The calculated sample size.
    """
    z = {0.90: 1.645, 0.95: 1.96, 0.99: 2.576}.get(confidence_level, 1.96)
    numerator = (z ** 2) * p * (1 - p)
    denominator = margin_of_error ** 2
    sample_size = numerator / denominator

    if population_size > 0:
        adjusted_sample_size = sample_size / (1 + (sample_size - 1) / population_size)
    else:
        adjusted_sample_size = sample_size
    
    return int(np.ceil(adjusted_sample_size))


def process_dataset(file_path: str, output_dir: str, filter_uber: bool = False, p: float = 0.5) -> None:
    """
    Processes a dataset: loads, filters, samples, and saves the result.

    Args:
        file_path (str): Path to the dataset file.
        output_dir (str): Directory to save the processed file.
        filter_uber (bool): Whether to filter for Uber rides (default is False).
        p (float): The proportion of variability for sampling (default is 0.5).
    """
    print(f"Processing file: {file_path}")
    
    # Load the dataset
    data = pd.read_parquet(file_path)
    
    # Apply filtering for Uber rides if needed
    if filter_uber:
        data = filter_uber_rides(data)
    
    # Determine population size
    population_size = len(data)
    print(f"Population size: {population_size}")
    
    # Calculate sample size
    sample_size = cochran_sample_size(population_size, confidence_level=0.95, margin_of_error=0.05, p=p)
    print(f"Calculated sample size: {sample_size}")
    
    # Sample the dataset
    sampled_data = data.sample(n=sample_size, random_state=42)
    
    # Save the sampled dataset
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    sampled_data.to_parquet(output_file)
    print(f"Processed file saved to: {output_file}")


# Example Workflow

# Input directories (where raw datasets are stored)
yellow_taxi_dir = "data/yellow_taxi"
hvhf_dir = "data/hvhf"

# Output directories (where processed datasets will be saved)
processed_yellow_taxi_dir = "processed_data/yellow_taxi"
processed_hvhf_dir = "processed_data/hvhf"

# Ensure output directories exist
os.makedirs(processed_yellow_taxi_dir, exist_ok=True)
os.makedirs(processed_hvhf_dir, exist_ok=True)

# Process Yellow Taxi datasets (using p = 0.5)
for file in os.listdir(yellow_taxi_dir):
    if file.endswith(".parquet"):
        process_dataset(
            file_path=os.path.join(yellow_taxi_dir, file),
            output_dir=processed_yellow_taxi_dir,
            filter_uber=False,
            p=0.5  # Higher variability for Yellow Taxi
        )

# Process HVFHV datasets (filter for Uber rides, using p = 0.4)
for file in os.listdir(hvhf_dir):
    if file.endswith(".parquet"):
        process_dataset(
            file_path=os.path.join(hvhf_dir, file),
            output_dir=processed_hvhf_dir,
            filter_uber=True,
            p=0.4  # Lower variability for Uber rides
        )


Processing file: data/hvhf\2019-02.parquet
Filtered Uber rides: 13504994 out of 20159102 total rides.
Population size: 13504994
Calculated sample size: 369
Processed file saved to: processed_data/hvhf\2019-02.parquet
Processing file: data/hvhf\2019-03.parquet
Filtered Uber rides: 17248340 out of 23864598 total rides.
Population size: 17248340
Calculated sample size: 369
Processed file saved to: processed_data/hvhf\2019-03.parquet
Processing file: data/hvhf\2019-04.parquet
Filtered Uber rides: 15657454 out of 21734822 total rides.
Population size: 15657454
Calculated sample size: 369
Processed file saved to: processed_data/hvhf\2019-04.parquet
Processing file: data/hvhf\2019-05.parquet
Filtered Uber rides: 15876126 out of 22329247 total rides.
Population size: 15876126
Calculated sample size: 369
Processed file saved to: processed_data/hvhf\2019-05.parquet
Processing file: data/hvhf\2019-06.parquet
Filtered Uber rides: 14397173 out of 21001990 total rides.
Population size: 14397173
Calc

### Load Taxi Zones

In [ ]:
def load_taxi_zones(shapefile):
    raise NotImplementedError()

In [ ]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    raise NotImplementedError()

### Calculate Sample Size

In [ ]:
def calculate_sample_size(population):
    raise NotImplementedError()

### Common Functions

In [ ]:
def get_all_urls_from_tlc_page(taxi_page):
    raise NotImplementedError()

In [ ]:
def filter_parquet_urls(all_urls):
    raise NotImplementedError()

### Process Taxi Data

In [ ]:
def get_and_clean_taxi_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [ ]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.contact(all_uber_dataframes)
    return uber_data

In [ ]:
def load_and_clean_uber_data():
    raise NotImplementedError()

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)